In [1]:
import gd
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

dataset = "mushrooms.txt" 
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]
n, d = X.shape
y = 2*y-3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

lambda_value = 0.01
M = np.max(np.linalg.norm(X_train, axis=1)) 
L = M**2 / 4 + lambda_value


In [5]:

def loss(w, X, y, lambda_value):
    yXw = y * (X @ w)
    log_loss = np.mean(np.log(1 + np.exp(-yXw)))
    reg_loss = (lambda_value / 2) * np.dot(w, w)
    return log_loss + reg_loss

def loss_grad(w, X, y, lambda_value):
    yXw = y * (X @ w)
    log_grad = np.mean((-y * X.T)/(1+np.exp(+yXw)), axis=1)
    reg_grad = lambda_value * w
    return log_grad + reg_grad

f = gd.Func(
    func=loss,
    grad=loss_grad
)

def batch_generator(X, y, batch_size):
    n_samples = X.shape[0]
    for i in range(0, n_samples, batch_size):
        yield (X[i:i + batch_size], y[i:i + batch_size])

workers = []
batch_size = 500
for X_batch, y_batch in batch_generator(X_train, y_train, batch_size):
    worker_func = gd.create_worker_func(f, X_batch, y_batch, lambda_value)
    workers.append(gd.Worker(worker_func))

def top_k(k, arr):
    new_arr = arr.copy()
    new_arr[arr < np.sort(arr)[-k]] = 0
    return new_arr

for worker in workers:
    worker.compress_op = lambda arr: top_k(5, arr)
    pass


dgd = gd.DistributedGD(workers, 2/(L+lambda_value))
w0 = np.zeros(d)  # Начальная инициализация весов
dgd.run(200, w0)  # Запускаем 100 итераций

# Строим график средней функции потерь
plt.plot([loss(w, X_train, y_train, lambda_value) for w in dgd.history])
plt.xlabel('Iteration')
plt.ylabel('Average Loss')
plt.title('Average Loss vs. Iteration')
plt.show()

In [3]:
A = np.array([[1,2], [3,4], [6,5]])
y = np.array([1, 1, -1])
A = A[:1]
y = y[:1]
np.mean([[1,2], [1,2], [1,5]], axis=0)

def top_k(k, arr):
    arr[arr < np.sort(arr)[-k]] = 0

arr = np.array([1, 2, 3, 4, 5])
top_k(2, arr)
arr

array([0, 0, 0, 4, 5])